# Tests of algorithms.tensorflow.weight_manager.py

In [1]:
from typing import List
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models

import weight_manager

In [2]:
# simple model creation

model = tf.keras.Sequential()
model.add(layers.Dense(units=10, input_shape=(1,)))
model.add(layers.Dense(units=10))
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32,32,3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                20        
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
Total params: 130
Trainable params: 130
Non-trainable params: 0
_________________________________________________________________


## Analysis of model.weights

In [28]:
model.weights 
# list of tf.Variable, 1 for each layer, therefore we use this python object

# model.weights returns :
# [layer0_kernel,layer0_bias, ..., layerN_kernel,layerN_bias]
# so parameters are the kernels and biases of each layer 

[<tf.Variable 'dense/kernel:0' shape=(1, 10) dtype=float32, numpy=
 array([[ 0.6097705 ,  0.97421175,  0.76544136,  0.04212335, -0.11383426,
          0.93249696,  0.98096126,  1.185389  ,  0.7979639 ,  0.6755691 ]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(10, 10) dtype=float32, numpy=
 array([[ 0.17526767,  0.99429166,  0.45228302, -0.0279308 ,  0.5181041 ,
          0.02135977,  0.3771191 ,  0.29630885,  0.4934162 ,  0.05509907],
        [ 0.36632243,  0.87222916,  0.7056887 ,  0.4357409 ,  0.74763453,
          1.0191514 ,  0.2789482 ,  0.77191526,  1.0342777 ,  0.14600462],
        [ 0.94309413,  0.8797434 ,  0.5216277 ,  0.76034737,  0.08473635,
          0.698376  ,  0.0564428 ,  0.5298964 ,  0.3333498 ,  0.8105746 ],
        [ 0.07442456,  0.753538  ,  0.92369866,  0.35825253,  0.77232677,
          0.79700994,  0.834941

In [11]:
var = tf.Variable(initial_value=np.full(shape=(10,), fill_value=1.), dtype='float32')

# it is important to force dtype float32 of weights updates

In [12]:
var.value()

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

Native methods for weights update

In [13]:
model.weights[1].assign_add(delta = var) # sums weight and delta

<tf.Variable 'UnreadVariable' shape=(10,) dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>

In [14]:
model.weights[1].assign_sub(delta = var) # sums weight and delta

<tf.Variable 'UnreadVariable' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

## Tests of weight_manager module

In [16]:
updates = [
    tf.Variable(initial_value=np.full(shape=(1,10), fill_value=1., dtype='float32')),
    tf.Variable(initial_value=np.full(shape=(10,), fill_value=1., dtype='float32')),
    tf.Variable(initial_value=np.full(shape=(10,10), fill_value=1., dtype='float32')),
    tf.Variable(initial_value=np.full(shape=(10,), fill_value=1., dtype='float32')),
]

# dtype='float32' is important

In [17]:
updates2 = [
    tf.Variable(initial_value=np.full(shape=(1,10), fill_value=2., dtype='float32')),
    tf.Variable(initial_value=np.full(shape=(10,), fill_value=2., dtype='float32')),
    tf.Variable(initial_value=np.full(shape=(10,10), fill_value=2., dtype='float32')),
    tf.Variable(initial_value=np.full(shape=(10,), fill_value=2., dtype='float32')),
]

In [18]:
weight_manager.increment_parameters(model=model, updates=updates, updates_multiplier=0.5)

In [19]:
model.weights # on bias layers we see clearly the results

[<tf.Variable 'dense/kernel:0' shape=(1, 10) dtype=float32, numpy=
 array([[ 0.6097705 ,  0.97421175,  0.76544136,  0.04212335, -0.11383426,
          0.93249696,  0.98096126,  1.185389  ,  0.7979639 ,  0.6755691 ]],
       dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32, numpy=array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(10, 10) dtype=float32, numpy=
 array([[ 0.17526767,  0.99429166,  0.45228302, -0.0279308 ,  0.5181041 ,
          0.02135977,  0.3771191 ,  0.29630885,  0.4934162 ,  0.05509907],
        [ 0.36632243,  0.87222916,  0.7056887 ,  0.4357409 ,  0.74763453,
          1.0191514 ,  0.2789482 ,  0.77191526,  1.0342777 ,  0.14600462],
        [ 0.94309413,  0.8797434 ,  0.5216277 ,  0.76034737,  0.08473635,
          0.698376  ,  0.0564428 ,  0.5298964 ,  0.3333498 ,  0.8105746 ],
        [ 0.07442456,  0.753538  ,  0.92369866,  0.35825253,  0.77232677,
          0.79700994,  0.834941

In [22]:
weight_manager.weighted_sum_parameters([updates, updates2], [0.5, 0.2])
# incompréhensible

[<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9], dtype=float32)>,
 <tf.Tensor: shape=(10, 10), dtype=float32, numpy=
 array([[0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
        [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([0.9, 0.9, 0.9, 0.9, 0.9, 0.9,

In [23]:
.5 * 1. + .2 * 2.

0.9

In [24]:
weight_manager.subtract_parameters(updates2, updates)

[<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>,
 <tf.Tensor: shape=(10, 10), dtype=float32, numpy=
 array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>]

In [25]:
weight_manager.add_parameters(updates2, updates)

[<tf.Tensor: shape=(1, 10), dtype=float32, numpy=array([[3., 3., 3., 3., 3., 3., 3., 3., 3., 3.]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.], dtype=float32)>,
 <tf.Tensor: shape=(10, 10), dtype=float32, numpy=
 array([[3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.]], dtype=float32)>,
 <tf.Tensor: shape=(10,), dtype=float32, numpy=array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3.], dtype=float32)>]

## Conclusion
primitives of weight_manager work with List[tf.Variable] objects 